In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Aluguel de Biciletas em WDC
    Prever quantas bicicletas serão alugadas por hora
        Base de treino - 01 a 19 de cada mês (02 anos-com respostas)
        Base teste - 20 ao final de cada mês (sem repostas)
    Métrica = Raiz quadrada de RMSE LOG
    
Passos
 1) Transformação da Base
 2) Dividir treino em treino + Validação
 3) Instalar o algoritmo modelo -> Base Treino
 4) Realizar as predições -> Base Validação
 5) Apurar a métrica
 6) Prever usando a base de teste
 7) Submeter ao Kaggle
 
 - Valores Contínuos -> Regressão
 - Valores Categóricos -> Classificação
 - Aprendizado Supervisionado
 - Overfitting (modelo viciado nos dados existentes e que é um "Vitinho" em novos dados)
 - Underfitting (modelo já é ruim no treino e nem chega a ser "Vitinho' nos novos dados)
 
 Olhar com carinho os códigos de python constantes do link abaixo:
 - https://www.kaggle.com/clperin/k20190916-1
    
 Nome alternativo para criar coluna -> feature engineering ("impressionar o chefe")
     fit - treinar o modelo - as árvores são preenchidas ficando em memória no modelo treinado de forma a ficar pronta para ser utilizado.
 predict - predizer o modelo
 
 https://valor.globo.com/brasil/noticia/2019/11/11/infraestrutura-estuda-fusao-entre-tres-estatais-da-area-de-transporte.ghtml
 
 How do you Interpret RMSLE (Root Mean Squared Logarithmic Error)?
 
 I've been doing a machine learning competition where they use RMSLE (Root Mean Squared Logarithmic Error) to evaluate the performance predicting the sale price of a category of equipment. The problem is I'm not sure how to interpret the success of my final result.

For example if I achieved a RMSLE of 1.052
could I raise it the the exponential power e and interpret it like rmse? (ie. e1.052=2.863=RMSE

)?

Could I then say that my predictions were ±$2.863
on average from the the actual prices? Or is there a better way to interpret the metric? Or can the metric even be interpreted at all with the exception of comparing to the other RMSLEs of other models? 



I haven't seen RMSLE before, but I'm assuming it's 1N∑Ni=1(log(xi)−log(yi))2−−−−−−−−−−−−−−−−−−−−−√

.

Thus exponentiating it won't give you RMSE, it'll give you

e1N∑Ni=1(log(xi)−log(yi))2√≠1N∑Ni=1(xi−yi)2−−−−−−−−−−−−−−√

.

If we take the log of both sides, we get the RMSLE versus 12log(1N∑Ni=1(xi−yi)2)

, which is clearly not the same thing.

Unfortunately, there isn't a good easy relationship in general (though someone smarter than me / thinking about it harder than me could probably use Jensen's inequality to figure out some relationship between the two).

It is, of course, the RMSE of the log-transformed variable, for what that's worth. If you want a rough sense of the spread of the distribution, you can instead get a rough sense of the spread of their logarithm, so that a RMSLE of 1.052 means that the "average" is 2.86
times as big as the true value, or 1/2.86. Of course that's not quite what RMSE means....

https://stats.stackexchange.com/questions/56658/how-do-you-interpret-rmsle-root-mean-squared-logarithmic-error

In [ ]:
df = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

In [ ]:
df.shape, test.shape

In [ ]:
# Verificando os dataframes
df.info()

In [ ]:
test.info()

datetime, season, holiday, workingday, weather, temp, atemp, humidity, windspeed

In [ ]:
# REalizando as trasnformações nos dados

In [ ]:
# Aplicar log na variável de resposta
df['count'] = np.log(df['count'])

In [ ]:
# Juntando os dataframente
df = df.append(test)


In [ ]:
df.info()

In [ ]:
# Converter a coluna datetime
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
# Criar Future Engineering(criar colunas) para o datetime
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

In [ ]:
# Separando os dataframes
test = df[df['count'].isnull()]

In [ ]:
# Excluindo os nulos do dataframe df
df = df[~df['count'].isnull()]

In [ ]:
df.shape, test.shape

In [ ]:
# Dividindo o dataframe de treino
# Importando o método scikitlearn para divisão
from sklearn.model_selection import train_test_split

In [ ]:
#Dividir a base de treino
train, valid = train_test_split(df, random_state=42)

In [ ]:
# verificando tamanhos
train.shape, valid.shape

In [ ]:
# Selecionando as colunas que iremos usar como entrada
# lista das colunas não usadas
removed_cols = ['casual', 'registered', 'count', 'datetime']

# Criar a lista das colunas de entrada
feats = [c for c in train.columns if c not in removed_cols]

In [ ]:
#Usando o modelo random forest
# Importando o modelo
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#Instanciar o modelo
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

In [ ]:
# Treinar o modelo. Romário não gostaria disso.
rf.fit(train[feats], train['count'])

In [ ]:
# Fazendo predições em cima dos dados de validação
preds = rf.predict(valid[feats])

In [ ]:
# verificando as previsões
preds

In [ ]:
# Verificando o real
valid['count'].head(3)

In [ ]:
# Vamos verificar o modelo com relação à métrica

# Importar a métrica
from sklearn.metrics import mean_squared_error

In [ ]:
# Aplicando a métrica
mean_squared_error(valid['count'], preds)**(1/2)

In [ ]:
# Vamos prever com base nos dados de treino
# Como o modelo se comporta prevendo em cima de dados desconhecidos?
train_preds = rf.predict(train[feats])

mean_squared_error(train['count'], train_preds)**(1/2)


In [ ]:
#Gerando as previsões para envio ao Kaggle

In [ ]:
# vamos fazer previsões para a base de teste
test['count'] = np.exp(rf.predict(test[feats]))

#mean_squared_error(test['count'], test_preds)**(1/2)


In [ ]:
# Gerando o arquivo para submeter ao kaggle
test[['datetime', 'count']].head(50)

In [ ]:
test[['datetime', 'count']].to_csv('rf.csv', index=False)